<a href="https://colab.research.google.com/github/Jotavenom/Proyecto-Bioinfo/blob/main/Fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# FASE 1: CONFIGURACIÓN Y
# ==========================================

print("⚙️ Paso 1: Instalando dependencias de sistema...")
# Instalamos MUSCLE (Software de alineamiento) y Biopython
!apt-get install muscle -qq > /dev/null
!pip install biopython -q

import os
from google.colab import files
from Bio import AlignIO, SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo

print("✅ Dependencias instaladas.\n")

# --- PASO 2: CARGA DE ARCHIVO FASTA ---
print("📂 Paso 2: Sube tu archivo Multi-FASTA (.fasta, .fna, .txt)")
print("   (Debe contener entre 5 y 15 secuencias de SVMPs)")

uploaded = files.upload()
filename = next(iter(uploaded)) # Detecta automáticamente el nombre del archivo

# Validación rápida
sequences = list(SeqIO.parse(filename, "fasta"))
print(f"\n🔍 Validación: Se detectaron {len(sequences)} secuencias en '{filename}'.")

if len(sequences) < 3:
    print("⚠️ ADVERTENCIA: Se recomiendan al menos 3 secuencias para un análisis informativo.")

# --- PASO 3: ALINEAMIENTO CON MUSCLE ---
output_aln = "alineamiento_muscle.aln"
print(f"\n🧬 Paso 3: Ejecutando algoritmo MUSCLE...")

try:
    # Configuramos la línea de comandos de MUSCLE
    # input: tu archivo crudo -> out: archivo alineado
    muscle_cline = MuscleCommandline(input=filename, out=output_aln)

    # Ejecutamos (stdout captura mensajes de éxito)
    stdout, stderr = muscle_cline()

    # Leemos el resultado para confirmar
    alignment = AlignIO.read(output_aln, "fasta")

    print(f"✅ ¡Éxito! Alineamiento generado en: '{output_aln}'")
    print(f"   Longitud del alineamiento: {alignment.get_alignment_length()} posiciones.")
    print(f"   Número de secuencias alineadas: {len(alignment)}")

    # --- PASO 4: GENERACIÓN DE CONSENSO (PREPARACIÓN PARA API) ---
    # Extraemos la secuencia consenso para usarla en la Fase 2 (InterPro)
    summary = AlignInfo.SummaryInfo(alignment)
    consensus = summary.dumb_consensus(threshold=0.6, ambiguous='X')

    print("\n📝 Vista previa de la Secuencia Consenso (Base para Fase 2):")
    print(str(consensus)[:80] + "...")

except Exception as e:
    print(f"❌ Error crítico durante el alineamiento: {e}")